In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import json
import concurrent.futures
import queue
from sympy import flatten
import time
from selenium.webdriver.support.ui import WebDriverWait
# Get today's date
current_date = datetime.today()


In [2]:
# # Install required packages
# import subprocess
# import sys

# def install_package(package):
#     """Install a package using pip"""
#     try:
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package])
#         print(f"✓ Successfully installed {package}")
#     except subprocess.CalledProcessError as e:
#         print(f"❌ Failed to install {package}: {e}")

# # List of required packages
# required_packages = [
#     "selenium",
#     "beautifulsoup4", 
#     "pandas",
#     "numpy",
#     "requests",
#     "sympy",
#     "openpyxl",  # for reading Excel files
#     "webdriver-manager",  # for automatic Chrome driver management
#     "mysql-connector-python",  # for MySQL database connection
#     "sqlalchemy"  # for database operations
# ]

# print("Installing required packages...")
# print("=" * 50)

# for package in required_packages:
#     install_package(package)

# print("\n" + "=" * 50)
# print("✓ Package installation completed!")
# print("\nNote: You may need to restart your kernel after installation.")
# print("Also ensure you have Chrome browser installed for Selenium to work.")

Installing required packages...
✓ Successfully installed selenium
✓ Successfully installed selenium
✓ Successfully installed beautifulsoup4
✓ Successfully installed beautifulsoup4
✓ Successfully installed pandas
✓ Successfully installed pandas
✓ Successfully installed numpy
✓ Successfully installed numpy
✓ Successfully installed requests
✓ Successfully installed requests
✓ Successfully installed sympy
✓ Successfully installed sympy
✓ Successfully installed openpyxl
✓ Successfully installed openpyxl
✓ Successfully installed webdriver-manager
✓ Successfully installed webdriver-manager
✓ Successfully installed mysql-connector-python
✓ Successfully installed mysql-connector-python
✓ Successfully installed sqlalchemy

✓ Package installation completed!

Note: You may need to restart your kernel after installation.
Also ensure you have Chrome browser installed for Selenium to work.
✓ Successfully installed sqlalchemy

✓ Package installation completed!

Note: You may need to restart your kerne

In [3]:
from urllib.parse import urlsplit, urlunsplit

def remove_query_from_url(url):
    url_parts = urlsplit(url)
    url_without_query = urlunsplit((url_parts.scheme, url_parts.netloc, url_parts.path, '', ''))
    return url_without_query



In [4]:
def generate_stubhub_url(artist):
    return "https://www.stubhub.ca/secure/search?q=" + artist


### Driver Interactions

In [5]:

def click_zones(driver):
    zones_div = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1s9c4ms-2.jFxikH')
    for zone in zones_div:
        if "zones" in zone.text.lower():
            zone.click()
            break
    
def click_checkboxes(driver):
    """
    Clicks each checkbox, unchecks all others, and returns the link each time.
    """
    # Find and click the "Zones" button to expand the filter options

    checkboxes = driver.find_elements(By.CSS_SELECTOR, 'input[type="checkbox"].sc-mhai9k-2.fOnHmE')
    links = []

    for checkbox in checkboxes:
        # Uncheck all checkboxes first
        for cb in checkboxes:
            if cb.is_selected():
                cb.click()
        checkbox.click()
        links.append(driver.current_url)
    if len(links) == 0:
        links.append(driver.current_url)
    return links

# Call the function and store the links




In [6]:
path = "../../Documents/Ticket Sales.xlsx"
unique_artists = pd.read_excel(path, sheet_name ="Events")["Artist"].unique()


### Cocunrrency Executors

In [7]:
# Initialize variables
event_listing = []
driver_queue = queue.Queue()
processed_titles = []
short_cols = ['id', 'eventId', 'Artist', 'Venue', 'City', 'Event Date', 'countryName', 'Event Name', 'Event Type', 'Performer Type', 'Performer', 'section', 'sectionId', 'sectionMapName', 'sectionType', 'row', 'rowId', 'seat', 'seatFrom', 'seatTo', 'faceValue', 'rawPrice', 'priceWithFees', 'price', 'ticketClass', 'ticketClassName', 'ticketTypeId', 'ticketTypeGroupId', 'listingTypeId', 'listingCurrencyCode', 'buyerCurrencyCode', 'faceValueCurrencyCode', 'Updated', 'formattedDealScore']

# Function to fetch artist data
def fetch_artist_data(artist):
    artist_search_url = generate_stubhub_url(artist)
    response = requests.get(artist_search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_grid = soup.find('script', type='application/json', string=lambda x: x and 'eventGrids' in x)
    if event_grid is None:
        return None
    json_str = event_grid.text.strip()
    data = json.loads(json_str)
    event_grid = data["eventGrids"]
    events = event_grid["0"]["items"][0:7]
    for event in events:
        event["Artist"] = artist
    return events

# Function to get listing info
def get_listing_info(url):
    response = requests.get(url)
    page_source = response.text
    soup = BeautifulSoup(page_source, 'html.parser')
    script_tag = soup.find('script', id='index-data', type='application/json')
    if script_tag is None:
        return None
    json_string = script_tag.string
    index_data = json.loads(json_string)
    grid_items = index_data['grid']['items']
    df = pd.DataFrame(grid_items)
    formatted_date = datetime.strptime(index_data["formattedEventDateTime"], '%a %b %d %Y %I:%M %p').date()
    df["Event Date"] = formatted_date
    df["Updated"] = pd.Timestamp.today().strftime('%Y-%m-%d')


    script_tag = soup.find('script', type='application/ld+json')
    if script_tag:
        event_data = json.loads(script_tag.string)
        df["Event Type"] = event_data["@type"]
        df["Performer Type"] = event_data["performer"][0]["@type"]
        df["Performer"] = event_data["performer"][0]["name"]
    
    df = df[df.columns.intersection(short_cols)]
    return df

# Function to attach event data
def attach_event_data(df, event):
    df["Venue"] = event["venueName"]
    df["Artist"] = event["Artist"]
    df["Event Name"] = event["name"]
    df["City"] = event["venueCity"]
    df["countryName"] = event['countryName']
    return df

# Function to get listing dataframe
def get_listing_df(checkbox_links, event):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        listings = list(executor.map(get_listing_info, checkbox_links))
    curr = pd.concat(listings, ignore_index=True)
    curr = attach_event_data(curr, event)
    event_listing.append(curr)

# Function to get checkbox links
def get_checkbox_links(event, listing_executor):
    # while True:
    #     if not driver_queue.empty():
    #         driver = driver_queue.get(block=True)
    #         break
    driver = driver_queue.get(block=True)

    try:
        url = event["url"]
        url = remove_query_from_url(url) + "?listingQty=&quantity=0"+"&betterValueTickets=false" + "&estimatedFees=false"
        driver.get(url)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Continue"]'))).click()
        except:
            pass
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Filters"]'))).click() # open filter menu
        click_zones(driver)
        checkbox_links = click_checkboxes(driver)
    except:
        checkbox_links = [url]
    listing_executor.submit(get_listing_df, checkbox_links, event)
    driver_queue.put(driver)

# Function to create driver
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    driver_queue.put(driver)
# Close drivers
def close_driver(driver):
    driver.quit()



# unique_artists = ['knock 2']
with concurrent.futures.ThreadPoolExecutor() as executor:
    events = list(executor.map(fetch_artist_data, unique_artists))
events = flatten(events)
events = list({v['eventId']: v for v in events if v is not None}.values())
events = [event for event in events if event['countryName'] in ['Canada', 'USA']]

# Get checkbox links
with concurrent.futures.ThreadPoolExecutor(10) as selenium_executor, concurrent.futures.ThreadPoolExecutor() as listing_executor, concurrent.futures.ThreadPoolExecutor() as driver_executor:
    selenium_executor.map(lambda _: create_driver(), range(8))
    selenium_executor.map(lambda x: get_checkbox_links(x, listing_executor), events)
    selenium_executor.shutdown(wait=True)
    driver_executor.map(close_driver, list(driver_queue.queue))
    listing_executor.shutdown(wait=True)

combined_df = pd.concat(event_listing, ignore_index=True)
combined_df.drop_duplicates(subset=['id', 'eventId'], keep='last', inplace=True)

combined_df


,id,eventId,section,sectionId,sectionMapName,sectionType,row,ticketClass,ticketClassName,rowId,...,Performer Type,Performer,Venue,Artist,Event Name,City,countryName,formattedDealScore,seatFrom,seatTo
0,9.782283e+09,158590245.0,General Admission,1745887.0,General Admission,4.0,,233.0,General Admission,2612382.0,...,MusicGroup,Seven Lions,Badlands Tent,seven lions,Badlands Music Festival with Seven Lions - Sat...,Calgary,Canada,NaN,NaN,NaN
1,-9.753093e+09,158590245.0,General Admission,1745887.0,,4.0,,233.0,General Admission,2612382.0,...,MusicGroup,Seven Lions,Badlands Tent,seven lions,Badlands Music Festival with Seven Lions - Sat...,Calgary,Canada,NaN,NaN,NaN
2,9.790113e+09,158590245.0,GA-Tier 3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,MusicGroup,Seven Lions,Badlands Tent,seven lions,Badlands Music Festival with Seven Lions - Sat...,Calgary,Canada,NaN,NaN,NaN
3,9.789877e+09,158590245.0,GA+-Tier 1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,MusicGroup,Seven Lions,Badlands Tent,seven lions,Badlands Music Festival with Seven Lions - Sat...,Calgary,Canada,NaN,NaN,NaN
4,9.789860e+09,158590245.0,GA+-Tier 3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,MusicGroup,Seven Lions,Badlands Tent,seven lions,Badlands Music Festival with Seven Lions - Sat...,Calgary,Canada,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15776,9.586825e+09,158118375.0,PREMIUM PARKING,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,MusicGroup,Seven Lions,Rawhide Western Town & Event Center,seven lions,Seven Lions,Chandler,USA,NaN,NaN,NaN
15777,9.781657e+09,158118375.0,General Admission,969276.0,General Admission,4.0,,233.0,General Admission,20547.0,...,MusicGroup,Seven Lions,Rawhide Western Town & Event Center,seven lions,Seven Lions,Chandler,USA,NaN,NaN,NaN
15778,-9.781657e+09,158118375.0,General Admission,969276.0,,4.0,,233.0,General Admission,20547.0,...,MusicGroup,Seven Lions,Rawhide Western Town & Event Center,seven lions,Seven Lions,Chandler,USA,NaN,NaN,NaN
15779,9.351102e+09,158118375.0,VIP,1593719.0,VIP,4.0,,582.0,VIP,1748848.0,...,MusicGroup,Seven Lions,Rawhide Western Town & Event Center,seven lions,Seven Lions,Chandler,USA,NaN,NaN,NaN


In [8]:
print(combined_df.columns)

Index(['id', 'eventId', 'section', 'sectionId', 'sectionMapName',
       'sectionType', 'row', 'ticketClass', 'ticketClassName', 'rowId',
       'rawPrice', 'price', 'ticketTypeId', 'ticketTypeGroupId',
       'listingTypeId', 'listingCurrencyCode', 'buyerCurrencyCode',
       'faceValueCurrencyCode', 'seat', 'faceValue', 'Event Date', 'Updated',
       'Event Type', 'Performer Type', 'Performer', 'Venue', 'Artist',
       'Event Name', 'City', 'countryName', 'formattedDealScore', 'seatFrom',
       'seatTo'],
      dtype='object')


In [9]:
import re

def clean_price_value(value):
    """
    Clean price values by removing currency symbols and converting to float.
    """
    if pd.isna(value) or value is None:
        return None
    str_value = str(value)
    cleaned = re.sub(r'[C$€£¥,\s]', '', str_value)
    try:
        return float(cleaned) if cleaned else None
    except ValueError:
        return None

print("Cleaning price columns...")



if 'price' in combined_df.columns:
    combined_df['price'] = combined_df['price'].apply(clean_price_value)
    print("✓ Cleaned price")

if 'faceValue' in combined_df.columns:
    combined_df['faceValue'] = combined_df['faceValue'].apply(clean_price_value)
    print("✓ Cleaned faceValue")

print(f"\nSample cleaned priceWithFees values:")


Cleaning price columns...
✓ Cleaned price
✓ Cleaned faceValue

Sample cleaned priceWithFees values:


In [11]:
combined_df.rename(columns={'Artist Name': 'Artist'}, inplace=True)

In [12]:
combined_df['Artist'] = combined_df['Artist'].astype(str).str.title()


## Database Integration

### Insert Scraped Data into CONCERT_SEATS Table

Now we'll insert the scraped concert seat data from `combined_df` into the database.

In [13]:
# Load database configuration and establish connection
import json
import subprocess
import sys

# Function to install missing packages
def install_missing_package(package):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ Successfully installed {package}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")
        return False

# Try to import required database packages, install if missing
try:
    import mysql.connector
    from mysql.connector import Error
    print("✓ mysql.connector imported successfully")
except ImportError:
    print("mysql-connector-python not found. Installing...")
    if install_missing_package("mysql-connector-python"):
        import mysql.connector
        from mysql.connector import Error
        print("✓ mysql.connector imported after installation")
    else:
        print("❌ Failed to install mysql-connector-python")

try:
    from sqlalchemy import create_engine
    print("✓ sqlalchemy imported successfully")
except ImportError:
    print("sqlalchemy not found. Installing...")
    if install_missing_package("sqlalchemy"):
        from sqlalchemy import create_engine
        print("✓ sqlalchemy imported after installation")
    else:
        print("❌ Failed to install sqlalchemy")

from urllib.parse import quote_plus

# Load database configuration
try:
    with open('db.json', 'r') as f:
        db_config = json.load(f)
    
    print("\nDatabase configuration loaded:")
    print(f"Host: {db_config['host']}")
    print(f"User: {db_config['user']}")
    print(f"Database: {db_config['database']}")
    print("Password: [HIDDEN]")
except FileNotFoundError:
    print("❌ db.json file not found. Please ensure it exists in the current directory.")
    db_config = None
except Exception as e:
    print(f"❌ Error loading database configuration: {e}")
    db_config = None

✓ mysql.connector imported successfully
✓ sqlalchemy imported successfully

Database configuration loaded:
Host: 192.168.68.74
User: root
Database: concert
Password: [HIDDEN]
✓ sqlalchemy imported successfully

Database configuration loaded:
Host: 192.168.68.74
User: root
Database: concert
Password: [HIDDEN]


In [14]:
# Establish MySQL connection
if db_config is None:
    print("❌ Cannot establish connection - database configuration not loaded")
    connection = None
    cursor = None
else:
    try:
        connection = mysql.connector.connect(**db_config)
        
        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"✅ Successfully connected to MySQL Server version {db_info}")
            
            cursor = connection.cursor()
            cursor.execute("SELECT DATABASE();")
            record = cursor.fetchone()
            print(f"You're connected to database: {record[0]}")
            
    except NameError:
        print("❌ mysql.connector not available. Please run the previous cell to install required packages.")
        connection = None
        cursor = None
    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")
        connection = None
        cursor = None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        connection = None
        cursor = None

✅ Successfully connected to MySQL Server version 5.5.5-10.6.21-MariaDB-ubu2004-log
You're connected to database: concert


C:\Users\eric9\AppData\Local\Temp\ipykernel_49012\2943637871.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()


In [15]:
# Prepare the combined_df for database insertion
print(f"Preparing to insert {len(combined_df)} rows into CONCERT_SEATS table...")
print(f"Combined DataFrame columns: {list(combined_df.columns)}")

# Ensure all required columns exist (add missing ones as None)
required_columns = [
    'id', 'section', 'sectionId', 'row', 'rowId', 'faceValue', 'rawPrice', 'priceWithFees',
    'eventId', 'sectionMapName', 'sectionType', 'seat', 'seatFrom', 'seatTo', 'ticketClass',
    'ticketClassName', 'price', 'ticketTypeId', 'ticketTypeGroupId', 'listingTypeId', 'City',
    'Event Date', 'countryName', 'Event Name', 'Event Type', 'Performer Type', 'Performer',
    'isFavorite', 'aggregateFavorites', 'listingCurrencyCode', 'buyerCurrencyCode',
    'faceValueCurrencyCode', 'Updated', 'isStanding', 'formattedFees', 'Artist', 'Venue',
    'formattedDealScore'
]

# Add missing columns with None values
for col in required_columns:
    if col not in combined_df.columns:
        combined_df[col] = None
        print(f"Added missing column: {col}")

# Rename 'row' to 'row_name' to match database schema
if 'row' in combined_df.columns:
    combined_df.rename(columns={'row': 'row_name'}, inplace=True)

# Ensure date columns are properly formatted
if 'Event Date' in combined_df.columns:
    combined_df['Event Date'] = pd.to_datetime(combined_df['Event Date'], errors='coerce').dt.date
    
if 'Updated' in combined_df.columns:
    combined_df['Updated'] = pd.to_datetime(combined_df['Updated'], errors='coerce')

print("✓ Data preparation completed")

Preparing to insert 15781 rows into CONCERT_SEATS table...
Combined DataFrame columns: ['id', 'eventId', 'section', 'sectionId', 'sectionMapName', 'sectionType', 'row', 'ticketClass', 'ticketClassName', 'rowId', 'rawPrice', 'price', 'ticketTypeId', 'ticketTypeGroupId', 'listingTypeId', 'listingCurrencyCode', 'buyerCurrencyCode', 'faceValueCurrencyCode', 'seat', 'faceValue', 'Event Date', 'Updated', 'Event Type', 'Performer Type', 'Performer', 'Venue', 'Artist', 'Event Name', 'City', 'countryName', 'formattedDealScore', 'seatFrom', 'seatTo']
Added missing column: priceWithFees
Added missing column: isFavorite
Added missing column: aggregateFavorites
Added missing column: isStanding
Added missing column: formattedFees
✓ Data preparation completed


In [16]:
from sqlalchemy.dialects.mysql import insert

# Insert data into CONCERT_SEATS table using SQLAlchemy (most efficient method)
try:
    # Build SQLAlchemy connection string
    user = db_config['user']
    password = db_config['password']
    host = db_config['host']
    database = db_config['database']
    
    # URL-encode the password to handle special characters
    encoded_password = quote_plus(password)
    connection_string = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}/{database}"
    
    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    # Prepare final column mapping to match database schema
    db_columns = {
        'id': 'id',
        'section': 'section', 
        'sectionId': 'sectionId',
        'row_name': 'row_name',
        'rowId': 'rowId',
        'faceValue': 'faceValue',
        'rawPrice': 'rawPrice',
        'priceWithFees': 'priceWithFees',
        'eventId': 'eventId',
        'sectionMapName': 'sectionMapName',
        'sectionType': 'sectionType',
        'seat': 'seat',
        'seatFrom': 'seatFrom', 
        'seatTo': 'seatTo',
        'ticketClass': 'ticketClass',
        'ticketClassName': 'ticketClassName',
        'price': 'price',
        'ticketTypeId': 'ticketTypeId',
        'ticketTypeGroupId': 'ticketTypeGroupId',
        'listingTypeId': 'listingTypeId',
        'City': 'city',
        'Event Date': 'event_date',
        'countryName': 'countryName',
        'Event Name': 'event_name',
        'Event Type': 'event_type',
        'Performer Type': 'performer_type',
        'Performer': 'performer',
        'isFavorite': 'isFavorite',
        'aggregateFavorites': 'aggregateFavorites',
        'listingCurrencyCode': 'listingCurrencyCode',
        'buyerCurrencyCode': 'buyerCurrencyCode',
        'faceValueCurrencyCode': 'faceValueCurrencyCode',
        'Updated': 'updated_date',
        'isStanding': 'isStanding',
        'formattedFees': 'formattedFees',
        'Artist': 'artist',
        'Venue': 'venue',
        'formattedDealScore': 'formattedDealScore'
    }
    
    # Select and rename columns to match database schema
    df_for_insert = combined_df.rename(columns=db_columns)
    
    # Perform true upsert (insert or update on duplicate key)
    batch_size = 1000
    table_name = 'CONCERT_SEATS'
    # Build the insert statement with ON DUPLICATE KEY UPDATE

    # Only use columns that exist in the DataFrame
    insert_cols = [col for col in df_for_insert.columns]
    update_cols = {col: getattr(insert(table_name).inserted, col) for col in insert_cols if col not in ['id']}  # Don't update primary key

    with engine.begin() as conn:
        for start in range(0, len(df_for_insert), batch_size):
            batch = df_for_insert.iloc[start:start+batch_size]
            stmt = insert(table_name).values(batch.to_dict(orient='records'))
            upsert_stmt = stmt.on_duplicate_key_update(**update_cols)
            conn.execute(upsert_stmt)
    
    print(f"✅ Successfully inserted {len(df_for_insert)} rows into CONCERT_SEATS table!")
    print(f"   Batch size: {batch_size}")
    print(f"   Database: {database}")
    
except Exception as e:
    print(f"❌ Error inserting data into database: {e}")
    print(f"   Make sure the CONCERT_SEATS table exists and has the correct schema.")
    
finally:
    # Close engine connection
    if 'engine' in locals():
        engine.dispose()
        print("Database connection closed.")

✅ Successfully inserted 15781 rows into CONCERT_SEATS table!
   Batch size: 1000
   Database: concert
Database connection closed.


In [17]:
# Verify the insertion by checking the database
try:
    if connection and connection.is_connected():
        # Count total rows in CONCERT_SEATS table
        cursor.execute("SELECT COUNT(*) FROM CONCERT_SEATS")
        total_rows = cursor.fetchone()[0]
        print(f"Total rows in CONCERT_SEATS table: {total_rows}")
        
        # Check for recent entries (today's date)
        cursor.execute("""
            SELECT COUNT(*) FROM CONCERT_SEATS 
            WHERE DATE(updated_date) = CURDATE()
        """)
        today_rows = cursor.fetchone()[0]
        print(f"Rows inserted today: {today_rows}")
        
        # Show sample of recently inserted data
        cursor.execute("""
            SELECT artist, venue, event_name, event_date, city, COUNT(*) as seat_count
            FROM CONCERT_SEATS 
            WHERE DATE(updated_date) = CURDATE()
            GROUP BY artist, venue, event_name, event_date, city
            ORDER BY seat_count DESC
            LIMIT 5
        """)
        
        recent_data = cursor.fetchall()
        if recent_data:
            print("\n📊 Sample of recently inserted data:")
            print("Artist | Venue | Event | Date | City | Seats")
            print("-" * 60)
            for row in recent_data:
                artist, venue, event, date, city, count = row
                print(f"{artist[:15]:<15} | {venue[:15]:<15} | {event[:15]:<15} | {date} | {city[:10]:<10} | {count}")
        
        print("\n✅ Database verification completed!")
        
except Error as e:
    print(f"❌ Error verifying database insertion: {e}")
    
finally:
    # Close MySQL connection
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

Total rows in CONCERT_SEATS table: 209476
Rows inserted today: 15781
Rows inserted today: 15781

📊 Sample of recently inserted data:
Artist | Venue | Event | Date | City | Seats
------------------------------------------------------------
Max Stlyer      | The Theatre at  | Max Amini       | 2025-10-04 | Toronto    | 41
Nao             | The Danforth Mu | Neema Naz       | 2025-08-09 | Toronto    | 41
Testpilot X Jau | Bugsys Cabaret  | X Burlesque     | 2025-07-12 | Las Vegas  | 29
Porter Robinson | Tinker Field    | EDC Orlando wit | 2025-11-07 | Orlando    | 22
Excision        | Fiddlers Green  | Excision        | 2025-08-02 | Englewood  | 22

✅ Database verification completed!
MySQL connection closed.

📊 Sample of recently inserted data:
Artist | Venue | Event | Date | City | Seats
------------------------------------------------------------
Max Stlyer      | The Theatre at  | Max Amini       | 2025-10-04 | Toronto    | 41
Nao             | The Danforth Mu | Neema Naz       | 2025-